# Ejemplo de uso de Censo 2017

La funcion de descarga borrara cualquier version anterior de la base de datos que exista. Es necesario ejecutarla una unica vez. Esto creara una base de datos DuckDB (SQL).

In [1]:
import censo2017
censo2017.descargar()

Borrando las versiones antiguas de la base que pudiera haber...
Descargando la base de datos desde GitHub...
Descomprimiendo los archivos necesarios...
Creando la base de datos...
Creando la tabla comunas...
Creando la tabla provincias...
Creando la tabla viviendas...
Creando la tabla variables...
Creando la tabla variables_codificacion...
Creando la tabla hogares...
Creando la tabla personas...
Creando la tabla zonas...
Creando la tabla regiones...


True

Una vez que se han importado las tablas, es posible leer selectivamente desde SQL. Para esto hay que abrir una conexion. Veamos, por ejemplo, la variable "p15" en la tabla "variables".

In [1]:
import duckdb
con = duckdb.connect(database = censo2017.archivo_sql())

# read the table variables in the connection and filter for the variable p15
con.execute("SELECT * FROM variables WHERE variable = 'p15'").df()


,tabla,variable,descripcion,tipo,rango
0,personas,p15,Nivel del Curso Más Alto Aprobado,integer,1 - 14


Tambien podemos ver la codificacion de la variable en la tabla "variables_codificacion".

In [9]:
con.execute("SELECT * FROM variables_codificacion WHERE variable = 'p15'").df()

,tabla,variable,valor,descripcion
0,personas,p15,1,Sala Cuna o Jardín Infantil
1,personas,p15,2,Prekínder
2,personas,p15,3,Kínder
3,personas,p15,4,Especial o Diferencial
4,personas,p15,5,Educación Básica
5,personas,p15,6,Primaria o Preparatoria (Sistema Antiguo)
6,personas,p15,7,Científico-Humanista
7,personas,p15,8,Técnica Profesional
8,personas,p15,9,Humanidades (Sistema Antiguo)
9,personas,p15,10,"Técnica Comercial, Industrial/Normalista (Sist..."


Para obtener informacion detallada de cada region en relación con las pregunta "p15", tenemos que pensar en los datos de REDATAM como un arbol, y es necesario unir “zonas” con “viviendas” por ID de zona, luego unir “viviendas” con “hogares” por ID de vivienda, y luego “hogares” con “personas” por ID de hogar. Esto se hace rápidamente con el backend DuckDB.

In [4]:
con.execute("""
SELECT "region", "p15", COUNT(*) AS "n"
FROM (SELECT "region", "geocodigo", "zonaloc_ref_id", "vivienda_ref_id", "LHS"."hogar_ref_id" AS "hogar_ref_id", "p15"
FROM (SELECT "region", "geocodigo", "zonaloc_ref_id", "LHS"."vivienda_ref_id" AS "vivienda_ref_id", "hogar_ref_id"
FROM (SELECT "region", "geocodigo", "LHS"."zonaloc_ref_id" AS "zonaloc_ref_id", "vivienda_ref_id"
FROM (SELECT SUBSTR(CAST("geocodigo" AS TEXT), 1, 2) AS "region", "geocodigo", "zonaloc_ref_id"
FROM "zonas") "LHS"
INNER JOIN (SELECT "zonaloc_ref_id", "vivienda_ref_id"
FROM "viviendas") "RHS"
ON ("LHS"."zonaloc_ref_id" = "RHS"."zonaloc_ref_id")
) "LHS"
INNER JOIN (SELECT "vivienda_ref_id", "hogar_ref_id"
FROM "hogares") "RHS"
ON ("LHS"."vivienda_ref_id" = "RHS"."vivienda_ref_id")
) "LHS"
INNER JOIN (SELECT "hogar_ref_id", "p15"
FROM "personas") "RHS"
ON ("LHS"."hogar_ref_id" = "RHS"."hogar_ref_id")
) "q01"
GROUP BY "region", "p15"
""").df()

,region,p15,n
0,01,10,2820
1,01,11,21102
2,01,12,44598
3,01,5,74514
4,01,8,52147
...,...,...,...
235,15,14,313
236,15,9,4081
237,15,10,2102
238,15,2,3643
